In [1]:
import numpy as np
import torch 
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris,load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers,IQPEmbedding
from pennylane.operation import Tensor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import matplotlib.pyplot as plt

np.random.seed(42)

In [24]:
X, y = load_diabetes(return_X_y=True)

# pick inputs and labels from the first two classes only,
# corresponding to the first 100 samples
X = X
y = y


for i in range(len(y)):
    if y[i] >= 185:
        y[i] = 0
    else:
        y[i] = 1
        

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
y_scaled = 2 * (y - 0.5)

X_new = SelectKBest(f_classif, k=2).fit_transform(X_scaled, y_scaled)


X_train, X_test, y_train, y_test = train_test_split(X_new, y_scaled)

In [25]:
def circuit(feature_vector,feature_vector_2,length):
#    qml.AngleEmbedding(features=feature_vector, wires=range(length),rotation='Z')
#    qml.adjoint(qml.AngleEmbedding(features=feature_vector_2, wires=range(length),rotation='Z'))
    qml.IQPEmbedding(features=feature_vector, wires=range(length),n_repeats=1)
    qml.adjoint(qml.IQPEmbedding(features=feature_vector_2, wires=range(length),n_repeats=1))
    return qml.probs(wires=range(length))
n_qubits = len(X_train[0])
dev_kernel = qml.device("default.qubit", wires=n_qubits)

In [26]:

@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """The quantum kernel."""
    u_1 = qml.matrix(circuit)(x1,x2,len(x1))
    u_2 = u_1.conjugate().transpose()
    projector = u_1+u_2
    return qml.expval(qml.Hermitian(projector,wires=range(n_qubits)))

In [27]:
def kernel_matrix_2(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[np.exp(-2 + kernel(a, b)) for b in B] for a in A])

In [28]:
def qsvm(Xtrain,ytrain):
    svm = SVC(kernel=new_kernel_par).fit(Xtrain, ytrain)
    return svm

In [72]:
def kernel_matrix_rectangular(A, B,i,gamma):
    #sigma=0.1
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    res = np.zeros((1,len(B)))
 #   for i in range(len(A)):
    for j in range(len(B)):
        # if ( i < j):
        res[0,j] = np.exp(-gamma**2*(2 - kernel(A[0], B[j]))**2)
         #   else:
             #   res[i,j] = res[j,i]
    return list(res[0])
def kernel_matrix_square(A, B, i,gamma):
    res = np.zeros((1,len(B)))
    for j in range(i+1,len(B)):
        res[0,j] = np.exp(-gamma**2*(2 - kernel(A[0], B[j]))**2)
    return list(res[0])

In [75]:
import time
import joblib
def new_kernel_par(A,B,gamma):
    ans = np.zeros((len(A),len(B)))
    s = time.time()
    # kernel_matrix(X_train[0], X_train)
    if len(A)==len(B):
        ans = joblib.Parallel(n_jobs=4)(joblib.delayed(kernel_matrix_square)([A[i]],B,i,gamma) for i in range(len(A)))
        #pool_obj.map(kernel_matrix,[(X_train[i],X_train) for i in range(len(X_train))])
        #print(ans)
        e = time.time()
        ans = np.array(ans)
        #print(ans.shape)
        ans=ans+ans.T+np.eye(len(A),len(B))
        #print(ans)
    else:
        ans = joblib.Parallel(n_jobs=4)(joblib.delayed(kernel_matrix_rectangular)([A[i]],B,i,gamma) for i in range(len(A)))
        #pool_obj.map(kernel_matrix,[(X_train[i],X_train) for i in range(len(X_train))])
        #print(ans)
        e = time.time()
        ans = np.array(ans)
        #print(ans.shape)
        #ans=ans+ans.T+np.eye(len(A),len(B))
        #print(ans)
   # ti=e-s
    #print((e-s)/(40)* 900*10**6/2 /3600/24)
    print(1)
    return ans   #, (e-s)/(400-20)* 900*10**6/2 /3600/24

In [79]:
from sklearn.model_selection import GridSearchCV
param_grid = {'gamma': [0.1, 1, 10]}
# Create an SVM classifier with a radial basis function (RBF) kernel
svm = SVC(kernel=new_kernel_par)
# Perform grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=4)
grid_search.fit(X_train,y_train)
best_params = grid_search.best_params_
print("Best Parameters:", best_params)
# Make predictions on the test data using the best model
y_pred = grid_search.predict(X_test)
#predictions = svm.predict(sxtes)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy for svm:", accuracy)

ValueError: 
All the 12 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_base.py", line 252, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_base.py", line 312, in _dense_fit
    X = self._compute_kernel(X)
        ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_base.py", line 513, in _compute_kernel
    kernel = self.kernel(X, self.__Xfit)
            

In [70]:
from sklearn.model_selection import GridSearchCV
param_grid = {'gamma': [0.3, 1, 10]}
# Create an SVM classifier with a radial basis function (RBF) kernel
svm = SVC(kernel=new_kernel_par)
# Perform grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=4)
grid_search.fit(X_train,y_train)
best_params = grid_search.best_params_
print("Best Parameters:", best_params)
# Make predictions on the test data using the best model
y_pred = grid_search.predict(X_test)
#predictions = svm.predict(sxtes)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy for svm:", accuracy)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Best Parameters: {'gamma': 0.3}
1
Accuracy for svm: 0.7927927927927928


In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
clf = SVC(kernel=new_kernel_par, C=1, random_state=42)
scores = cross_val_score(clf, X_new, y_scaled, cv=5)
scores

In [ ]:
np.mean(scores)

In [38]:
selector.get_support()

array([False, False,  True, False, False, False, False,  True,  True,
        True])

In [69]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
clf2 = svm.SVC(kernel='rbf', C=1, random_state=42)
scores2 = cross_val_score(clf2, X_scaled, y_scaled, cv=5)
scores2

array([0.78651685, 0.80898876, 0.79545455, 0.82954545, 0.79545455])

In [11]:
from sklearn import svm
from sklearn.metrics import confusion_matrix

In [50]:
svc = svm.SVC(kernel='rbf').fit(X_train, y_train)
predictions = svc.predict(X_test)
confusion_matrix(y_test, predictions)


array([[21, 17],
       [ 7, 66]])

In [51]:
svc = svm.SVC(kernel=new_kernel_par).fit(X_train, y_train)
predictions = svc.predict(X_test)
confusion_matrix(y_test, predictions)

1
1


array([[20, 18],
       [ 5, 68]])